# Bilibili Converter
## 1.爬取数据

In [2]:
import requests,json,time
def loadJson(path):
    with open(path, 'r', encoding='utf-8') as f:
        fileData = json.load(f)
    return fileData

def writeJson(name,path,isFormat=True):
    with open(path, 'w', encoding='utf-8') as f:
        if isFormat:
            f.write(json.dumps(name,ensure_ascii=False,indent=2))
        else:
            f.write(json.dumps(name))
    return 

这里可以得到某个频道的所有播放大于5万的历史视频。   
但是你需要提供该频道的id和你自己得到的offset。  
具体的方式是：打开你需要的频道，F12查看api请求，找到下面对应的api即可知道所需的offset（不填offset能不能用没测试过）频道id直接在url就可以看到

In [1]:
allVideo = []
offset = "428583780_1658362131"
id = "1393143"
for i in range(500):
    url = "https://api.bilibili.com/x/web-interface/web/channel/multiple/list?channel_id={}&sort_type=hot&offset={}&page_size=30".format(id,offset)
    time.sleep(0.5)
    data = requests.get(url=url).json()
    allVideo.extend(data['data']['list'])
    offset = data['data']['offset']
    if i % 10 == 0:
        print(i)
    if not data['data']['has_more']:
        break
   

NameError: name 'time' is not defined

保存你爬取到的频道视频信息。

In [20]:
writeJson(allVideo,"allvideo.json")

由于视频信息不够完整，我们需要得到一些视频的基础信息，这里要用到另一个api读取基础信息，以及访问网页本体得到时间信息和tag信息。可根据自己需要选取，

In [4]:
from bs4 import BeautifulSoup
baseInfoList = []
timeList = []
tagList = []
dic_header = {'User-Agent':'Mozilla/5.0'}
def get_base_info(aid):
    base_info_url = f'https://api.bilibili.com/x/web-interface/archive/stat?aid={aid}'
    base_info = {}
    try:
        base_info = requests.get(base_info_url,headers=dic_header).json()['data']
        #print('播放量：{}\n弹幕数量：{}\n收藏数量：{}\n硬币数量：{}\n分享数量：{}\n点赞数量：{}\n-----\n评论数量：{}'.format(
    except:
        print('Error')
    return base_info
def get_title_time_tag(bvid):
    tag_lst = []
    url = f'https://www.bilibili.com/video/{bvid}'
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')
    tags = soup.find_all('a',class_='tag-link')
    title_lst = soup.find_all('title')
    title = title_lst[0].text.split('_')[0]
    time_lst = soup.find_all(itemprop='uploadDate')
    if(len(time_lst)>0):
        time = time_lst[0]['content'].split(' ')[0]
    else:
        time = "error"
    for tag in tags:
        tag_lst.append(tag.text.strip())
    return title,time,list(set(tag_lst))


In [42]:
newVideoList = []
errorList = []
count = 0

In [10]:
allVideo = loadJson("newAllVideo.json")
recordCount = 0 #如果你中途断掉了，可以改recordCount到你断掉的点，继续进行下载
count = 0
import time,random
for i in range(len(allVideo)):
    if count >= recordCount:      
        baseInfo = get_base_info(allVideo[i]['metaData']['id'])
        time.sleep(1+random.random())
        
        if(type(baseInfo)==None):
            errorList.append(allVideo[i])  
        allVideo[i]['baseInfo']=baseInfo  
        if count % 100 == 0:
            print(count)
    count+=1

2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100


In [12]:
writeJson(allVideo,"newAllVideo2.json") #保存完整的视频信息

## 数据转化
将原始json数据转化为可用的csv
### 时间顺序显示视频

In [18]:
import csv
f = open("./allVideo.csv","w",encoding="utf-8-sig",newline='')
writer = csv.writer(f)
line = ['name','type','value','date']
writer.writerow(line)
for video in allVideo:
    line = []
    if video['baseInfo'] != None:
        if video['baseInfo']['copyright']==1:
            line = [video['metaData']['name'],video['metaData']['author_name'],video['baseInfo']['view'],video['time']]
            # csv.writer()中可以传一个文件对象
            writer.writerow(line)

### 根据视频信息制作作者排行榜

In [38]:
import datetime
allVideo = loadJson("newAllVideo2.json")
allAuthor = []
allAuthorCount = {}
for video in allVideo:
    try:
        if video['metaData']['author_name'] not in allAuthor and video['baseInfo']['copyright']==1:
            allAuthor.append(video['metaData']['author_name'])
            allAuthorCount[video['metaData']['author_name']]=1
        elif video['baseInfo']['copyright']==1:
            allAuthorCount[video['metaData']['author_name']]+=1
    except:
        print(video)


{'metaData': {'id': 51929624, 'name': '【Undertale AU】各路审判曲完整版合集（2.26更新到123P）', 'cover': 'http://i1.hdslb.com/bfs/archive/0187d9cb95ae138caacec4ef2f63a383aeccfcbe.jpg', 'view_count': '31.6万', 'like_count': '7220', 'duration': '7:02:20', 'author_name': '永神ヽ幽鸾', 'author_id': 397236985, 'bvid': 'BV1P4411Y7BD', 'danmaku': 6771, 'card_type': 'archive', 'sort': 'hot', 'filt': 0}, 'baseInfo': None, 'tag': [], 'time': ''}
{'metaData': {'id': 27089874, 'name': '【中文字幕】【undertale音乐剧】狂妄之人-最终の审判', 'cover': 'http://i1.hdslb.com/bfs/archive/4ecfe14b22606a9bb098c57ef77ff514d3c511eb.jpg', 'view_count': '39.7万', 'like_count': '8183', 'duration': '7:25', 'author_name': '账号已注销', 'author_id': 87148129, 'bvid': 'BV1As411E7tU', 'danmaku': 650, 'card_type': 'archive', 'sort': 'hot', 'filt': 0}, 'baseInfo': None, 'tag': [], 'time': ''}
{'metaData': {'id': 27019526, 'name': 'Undertale 传说之下 Megalovania Toby Fox [piano]', 'cover': 'http://i1.hdslb.com/bfs/archive/f85bfaab2f93267cfd07fd291e03ee805bd182b4.jpg', 'vie

In [102]:
filterAuthor = [] # 这里筛选掉视频少于一定数量的作者，减少非活跃的作者，避免出现”圈外人“。以及去掉难以进入排行榜的作者，加快计算速度。
for k,v in allAuthorCount.items():
    if v > 2 and k != '账号已注销':
        filterAuthor.append(k)
print(len(filterAuthor))

204


In [103]:
# 得到所有筛选过作者的视频，并排序得到最小时间和最大时间（因为原js脚本没有插值功能）
viewDict = {}
timeList = []
for name in filterAuthor:
    viewDict[name]=[]
for video in allVideo:
    if video['metaData']['author_name'] in filterAuthor:
        try:
            if(video['baseInfo']['copyright']==1):
                unit = (video['baseInfo']['view'],datetime.datetime.strptime(video['time'],"%Y-%m-%d").timestamp())
                viewDict[video['metaData']['author_name']].append(unit)
                timeList.append(datetime.datetime.strptime(video['time'],"%Y-%m-%d").timestamp())
        except:
            print("这条视频没有时间信息？:"+video)

timeList.sort()


{'metaData': {'id': 723291572, 'name': '【明日方舟新春会】感染者之盾', 'cover': 'http://i2.hdslb.com/bfs/archive/8bf8d6a3ed02caa4319bed5cc90968d5f9b491b1.jpg', 'badge_title': '精选', 'badge_background': 'http://i0.hdslb.com/bfs/tag/14f32b8a0f0002d16fdd57d3b8ddaf9e50f3c5fe.png', 'view_count': '66.8万', 'like_count': '5.6万', 'duration': '7:14', 'author_name': '阴险之鹰', 'author_id': 357423129, 'bvid': 'BV1VS4y1o7TV', 'danmaku': 915, 'card_type': 'archive', 'sort': 'hot', 'filt': 0}, 'baseInfo': {'aid': 723291572, 'bvid': 'BV1VS4y1o7TV', 'view': 668180, 'danmaku': 915, 'reply': 1029, 'favorite': 31388, 'coin': 48519, 'share': 8064, 'like': 55779, 'now_rank': 0, 'his_rank': 0, 'no_reprint': 1, 'copyright': 1, 'argue_msg': '', 'evaluation': ''}, 'tag': [], 'time': 'error'}


In [98]:
# 将视频播放量视为增量，转化为总量
endTime=timeList[-1]
for k,v in viewDict.items():
    viewDict[k]=sorted(v,key=lambda v : v[1])
    totalView = 0
    for i in range(len(viewDict[k])):
        viewDict[k][i]=list(viewDict[k][i])
        viewDict[k][i][0]+=totalView
        totalView=viewDict[k][i][0]
    viewDict[k].append((viewDict[k][-1][0],endTime))



In [100]:
# 进行插值处理
import numpy as np
viewDictF = {}

isFirst = True
for k,v in viewDict.items():
    indexIn = []
    valueIn = []
    startTime = datetime.datetime.fromtimestamp(v[0][1])
    endTime = datetime.datetime.fromtimestamp(v[-1][1])
    
    dayLen = (endTime-startTime).days+1
    viewDictF[k]={}
    viewDictF[k]['start']=startTime
    viewDictF[k]['timeSeries']=np.full(dayLen,np.nan)
    indexAll = [i for i in range(dayLen)]
    for item in v:
        index = (datetime.datetime.fromtimestamp(item[1])-startTime).days
        viewDictF[k]['timeSeries'][index]=item[0]
        indexIn.append(index)
        valueIn.append(item[0])
    #print(indexIn)
    indexOut = list(set(indexAll)-set(indexIn))
    interP = np.interp(indexOut, indexIn, valueIn)
    viewDictF[k]['timeSeries'][indexOut]=interP
    

In [101]:
# 存储作者排行榜的csv
import csv
f = open("./allAuthor.csv","w",encoding="utf-8-sig",newline='')
writer = csv.writer(f)
line = ['name','type','value','date']
writer.writerow(line)
for k,v in viewDictF.items():
    for i in range(v['timeSeries'].shape[0]):
        line = []
        line = [k,k,v['timeSeries'][i],(v['start']+datetime.timedelta(days=i)).strftime("%Y-%m-%d")]
        # csv.writer()中可以传一个文件对象
        writer.writerow(line)